# Setup Variables and home directory

MNIST, FashionMNIST, GTSRB, Cifar10

In [1]:
# set homw directory
import os
from pathlib import Path

base = Path().cwd()

if base.name != 'runtime-monitoring':
    os.chdir('../')
    base = Path().cwd()

base

PosixPath('/home/ah19/runtime-monitoring')

# Libraries

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import time
from fastprogress import progress_bar

import torch
import torchvision
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchinfo import summary

cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

In [3]:
from utilities.utils import *
from utilities.scaleFunctions import *
from utilities.pcaFunctions import *
from utilities.pathManager import fetchPaths
from utilities.MonitorBDD import MonitorBDD

In [4]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

### Setup Variables

In [5]:
SEED = 42
CUDA = 0
GPU_NAME = f'cuda:{CUDA}'

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

device = get_device(GPU_NAME)
torch.cuda.get_device_name(device)

'NVIDIA A100 80GB PCIe'

# Paths

In [6]:
DATASET = 'MNIST'
DATASET = 'FashionMNIST'
DATASET = 'GTSRB'

LHLs = {
    'MNIST': 60,
    'FashionMNIST': 30,
    'GTSRB': 50
}


LHL = LHLs[DATASET]

In [7]:
paths = fetchPaths(base, DATASET, '', False)

path_data = paths['data']
path_lhl = paths['lhl']

path_saved_model = paths['saved_models'].parent

configs = load_json(paths['configuration'])
config = configs['configuration']
model_setup = configs['model_setup']
model_config = configs['model_config']
optim_name = list(config['optimizer'].keys())[0]
optim_args = config['optimizer'][optim_name]
scheduler_name = list(config['scheduler'].keys())[0]
scheduler_args = config['scheduler'][scheduler_name]
batch_size = model_config['batch_size']

In [8]:
POSTFIX = f'{optim_name}-{batch_size}-{LHL}'
POSTFIX += '_selected_classes'
FILENAME_POSTFIX = f'{DATASET}_{POSTFIX}'

FILENAME_POSTFIX

'GTSRB_AdamW-32-50_selected_classes'

In [9]:
saved_model = list(path_saved_model.glob(f"{FILENAME_POSTFIX}*.pth.tar"))

if len(saved_model) > 0: saved_model = saved_model[0]
else: saved_model = None
    
saved_model

PosixPath('/home/ah19/runtime-monitoring/experiments/GTSRB/saved-models/GTSRB_AdamW-32-50_selected_classes-acc-0.96-loss-0.163.pth.tar')

# Import Data

In [10]:
from models.mnist_model import MNIST_Model
from models.fashionmnist_model import FashionMNIST_CNN
from models.gtsrb_model import GTSRB_CNN

from models.transformers import transformers

models = {
    'mnist': MNIST_Model,
    'fashionmnist': FashionMNIST_CNN,
    'gtsrb': GTSRB_CNN
}

model_ = models[DATASET.lower()]
transformer = transformers[DATASET.lower()]

In [11]:
feature_names = get_labels(DATASET)

train_data = get_dataset(DATASET, path_data, train=True, transform=transformer['train'])
test_data = get_dataset(DATASET, path_data, train=False, transform=transformer['test'])
len(train_data), len(test_data)

(64000, 4224)

In [12]:
skip_classes = [2, 5, 35, 40]

new_classes = np.setdiff1d( np.array(list(feature_names.keys())), skip_classes )

class_map = { c:i for i, c in enumerate(new_classes)}
class_map_rev = {v:k for k,v in class_map.items()}


def collate_fn(batch):
    id_drop = []
    for i, (_, y) in enumerate(batch):
        if y in skip_classes:
            id_drop.append(i)
    
    for i in id_drop[::-1]:
        batch.pop(i)
    
    x = torch.stack([x for x,_ in batch])
    y = torch.tensor([y for _,y in batch])
    
    y.apply_(lambda c: class_map[c])
    
    return x, y

In [13]:
trainloader = get_dataLoader(train_data, model_config['batch_size'], True, collate_fn=collate_fn)
testloader = get_dataLoader(test_data, model_config['batch_size'], False, collate_fn=collate_fn)

In [14]:
def create_model(lhl, num_classes):
    
    # model
    model_setup['last_hidden_neurons'] = lhl
    
    # torch 2.0 compile and parallel data training
    model = model_(**model_setup, outneurons = num_classes).to(device)
    model = torch.compile(model)
    nn.DataParallel(model, device_ids=[CUDA])
    
    # loss function
    loss_function = nn.CrossEntropyLoss()
    
    # optimizer and scheduler
    optimizer = getattr(torch.optim, optim_name)(model.parameters(), lr=model_config['lr'], **optim_args)
    scheduler = getattr(torch.optim.lr_scheduler, scheduler_name)(optimizer, **scheduler_args)
        
    return model, loss_function, optimizer, scheduler

In [15]:
if DATASET=='MNIST': model_config['epochs'] = 3
if DATASET=='GTSRB': model_config['epochs'] = 8

In [16]:
def start_training_testing(model_name, model, loss_function, optimizer, scheduler):
    # training testing attributes
    kwargs = {
        'model': model,
        'loss_function': loss_function,
        'optimizer': optimizer,
        'lr_scheduler': scheduler,
        'device': device,
        'model_path': path_saved_model / f"{model_name}.pth.tar",
        'trainloader': trainloader,
        'testloader': testloader,
        'config': model_config
    }

    # run training testing
    return run_training_testing(**kwargs)

In [17]:
model, loss_function, optimizer, scheduler = create_model(LHL, new_classes.shape[0])

if saved_model is None:
    (train_losses,
    train_accs,
    test_losses,
    test_accs,
    train_loss,
    train_acc,
    test_loss,
    test_acc,
    confusion_matrix_train,
    confusion_matrix_test,
    saved_model) = start_training_testing(FILENAME_POSTFIX, model, loss_function, optimizer, scheduler)

In [18]:
load_checkpoint(model, saved_model)

In [19]:
trainloader = get_dataLoader(train_data, model_config['batch_size'], True)
testloader = get_dataLoader(test_data, model_config['batch_size'], False)

In [20]:
def get_logits(loader):
    df = pd.DataFrame()

    with torch.no_grad():

        for x, y in progress_bar(loader):

            # extract last hidden layer and predicted class
            logits, y_pred = model.output_last_layer(x.to(device))
            
            y_pred = y_pred.argmax(dim=1).cpu()

            temp_df = pd.DataFrame(logits.cpu().numpy())
            temp_df['y'] = y.cpu().numpy()
            temp_df['true'] = temp_df['y'] == y_pred.apply_(lambda c: class_map_rev[c]).numpy()

            df = pd.concat([df, temp_df])

    return df

In [21]:
df_train = get_logits(trainloader)

In [22]:
df_test = get_logits(testloader)

In [61]:
df_train.shape, df_test.shape

((64000, 52), (4224, 52))

In [23]:
# select only true classified
# df_true = df_train.copy()
df_true = pd.concat([df_train[df_train["true"] == True].copy(), df_test[df_test["true"] == True].copy()])
df_true = df_true.drop("true", axis=1).reset_index(drop=True)

# df_true = df_train[df_train["true"] == True].copy()
# df_true = df_true.drop("true", axis=1).reset_index(drop=True)

df_true.shape

(58979, 51)

In [24]:
# fit scaler and pca
scaler_ = fitStandardScalerSingle(df_true, LHL)
pca_ = fitPCASingle(df_true, scaler=scaler_, numNeurons=LHL)

In [25]:
df_train_pca = applyPCASingle(df_train, scaler_, pca_, numNeurons=LHL)
df_test_pca = applyPCASingle(df_test, scaler_, pca_, numNeurons=LHL)
df_true_pca = applyPCASingle(df_true, scaler_, pca_, numNeurons=LHL)

In [26]:
gte_mean, top_third = neuronsLoadingsSingle(pca_, numNeurons=LHL, var_thld=0.9, loadings_thld=0.5)

# Build BDD

In [27]:
# cudd.BDD()

In [83]:
# each set of data has its own threshold
thld_p = 0
eta = 0

neurons = []
# neurons = gte_mean

# if flavor=='pca':
#     nerons = [f'x{i}' for i in range(numComponents(pca))]
#     subset_name = 'components'

# calculate threshold
if thld_p == 0:
    thld = np.zeros(df_true.drop('y', axis=1).shape[1])
else: thld = np.quantile( df_true.drop('y', axis=1), thld_p, axis=0)

# thld = np.mean( df_true.drop('y', axis=1), axis=0)

st = time.perf_counter()

df_train_cls = df_train.loc[~df_train['y'].isin(skip_classes)].copy()
df_test_cls = df_test.loc[~df_test['y'].isin(skip_classes)].copy()
df_eval = pd.concat([df_train.loc[df_train['y'].isin(skip_classes)].copy(),
                     df_test.loc[df_test['y'].isin(skip_classes)].copy()])

# BDD
patterns = MonitorBDD( df_true.shape[1]-1, thld, neurons=neurons, reorder=False, memory=10)
df_train_cls_copy, df_test_cls_copy, df_eval = patterns.add_dataframe( df_true, eta, eval_dfs=[
    df_train_cls, df_test_cls, df_eval] )

en = round(int(time.perf_counter() - st) / 60, 3)

# score
df_train_scores = patterns.score_dataframe_multi_eta(df_train_cls_copy, eta)
df_test_scores = patterns.score_dataframe_multi_eta(df_test_cls_copy, eta)
df_eval_scores = patterns.score_dataframe_multi_eta(df_eval, eta)

# add metadata
patterns.stats['num_observations'] = df_true.shape[0]

en

0.567

In [84]:
patterns.stats

,thld,eta,build_time_min,size_mb,reorder_time_min,num_patterns,num_unique_patterns_%,num_reorder,num_neurons,start_time,end_time,num_observations
1,NaN,0.0,0.183,214.6,0.0,58979.0,96.1,0.0,50.0,2023-09-16 13:15:35,2023-09-16 13:15:47,58979


In [85]:
df_train_scores.loc[ df_train_scores['y'] == 'all']

,y,total_count,total_misclassified,unrecognized,unrecognized_and_misclassified,unrecognized_and_classified,NPR,NPV,specificity,eta,subset_name,num_neurons
39,all,58140.0,2805.0,2805.0,2805.0,0.0,0.048246,1.0,1.0,0,None,50.0


In [86]:
df_test_scores.loc[ df_test_scores['y'] == 'all']

,y,total_count,total_misclassified,unrecognized,unrecognized_and_misclassified,unrecognized_and_classified,NPR,NPV,specificity,eta,subset_name,num_neurons
39,all,3853.0,209.0,206.0,206.0,0.0,0.053465,1.0,0.985646,0,None,50.0


In [87]:
df_eval_scores.loc[ df_eval_scores['y'] == 'all']

,y,total_count,total_misclassified,unrecognized,unrecognized_and_misclassified,unrecognized_and_classified,NPR,NPV,specificity,eta,subset_name,num_neurons
4,all,6231.0,6231.0,6223.0,6223.0,0.0,0.998716,1.0,0.998716,0,None,50.0


In [88]:
df_eval_scores

,y,total_count,total_misclassified,unrecognized,unrecognized_and_misclassified,unrecognized_and_classified,NPR,NPV,specificity,eta,subset_name,num_neurons
0,2,1573.0,1573.0,1572.0,1572.0,0.0,0.999364,1.0,0.999364,0,None,50.0
1,5,1497.0,1497.0,1492.0,1492.0,0.0,0.996660,1.0,0.996660,0,None,50.0
2,35,1575.0,1575.0,1573.0,1573.0,0.0,0.998730,1.0,0.998730,0,None,50.0
3,40,1586.0,1586.0,1586.0,1586.0,0.0,1.000000,1.0,1.000000,0,None,50.0
4,all,6231.0,6231.0,6223.0,6223.0,0.0,0.998716,1.0,0.998716,0,None,50.0
